# Complete Neuron

## Perameters

- `learning_rate`
- `activation`
- `loss`
- `optimizer`

In [1]:
#
# Imports
#

import numpy as np
from collections.abc import Iterable
from numbers import Number

In [2]:
#
# Customer Exceptions
#

class InvalidInitialWeights(Exception): pass

class UnexpectedInputsShape(Exception): pass
class UnexpectedOutputsShape(Exception): pass

class InvalidActivation(Exception): pass
class InvalidOptimizer(Exception): pass
class InvalidLoss(Exception): pass

In [3]:
#
# Sigmoid Activation
#

class Sigmoid():

  def apply(self, x):
    return 1 / (1 + np.exp(-x))
  
  def derivative(self, x):
    sig = self.apply(x)
    return sig * (1 - sig)

In [4]:
#
# Mean Squared Error
#

class MeanSquaredError():
  N = 0

  def calculate(self, y_true:Iterable, y_pred:Iterable):
    self.N = len(y_true)
    return np.mean(np.square(y_true - y_pred))
  
  def derivative(self, y_true:Number, y_pred:Number):
    return -2*(y_true-y_pred)/self.N

In [39]:
#
# Stochastic Gradient Descent
#

class StochasticGradientDescent():
  MODES = {
    'SAMPLE': 1,
    'BATCH': 2,
  }

  def __init__(self, learning_rate=None):
    self.learning_rate = learning_rate
    self.mode = self.MODES['SAMPLE']

  def update(self, model, learning_rate=0.01):
    # learning rate
    lr = self.learning_rate or model.learning_rate or learning_rate

    # hierarchy functions
    def update_neuron(neuron, lr): neuron.update(lr)
    def update_layer(layer, lr):
      for neuron in layer.neurons: update_neuron(neuron, lr)

    # if ann
    if hasattr(model, 'layers'):
      for layer in model.layers: update_layer(layer, lr)
    
    # if layer
    elif hasattr(model, 'neurons'):
      for neuron in model.neurons: update_neuron(neuron, lr)
    
    # if neuron
    elif hasattr(model, 'forward'):
      update_neuron(model, lr)



In [49]:
class Neuron():

  # 
  # Props
  # 

  builtin:dict = {
    'activation': {
      'default': Sigmoid,
      'sigmoid': Sigmoid,
    },
    'loss_function': {
      'default': MeanSquaredError,
      'sigmoid': MeanSquaredError,
    },
    'optimizer': {
      'default': StochasticGradientDescent,
      'sgd': StochasticGradientDescent,
    }
  }

  errors:dict = {
    'activation':    InvalidActivation,
    'loss_function': InvalidLoss,
    'optimizer':     InvalidOptimizer,
  }

  input:Iterable = []
  output:float = 0.0

  delta:float = 0.0


  weight_gradient:Iterable = []
  bias_gradient:float = 0.0



  # 
  # Constructor
  # 

  def __init__(
      self,

      input_size:int,
      
      learning_rate:float=0.1,
      activation:str|object='default',
      loss_function:str|object='default',
      optimizer:str|object='default',

      initial_weights:Number|Iterable=0,
      initial_bias:Number=0,
    ):

    #> input size and learning rate

    # Input Size
    self.input_size = input_size
    self.input = np.repeat(0, self.input_size)
    
    # Learning Rate
    self.learning_rate = learning_rate

    #> activation, loss_function and optimizer

    params = {
      'activation':    activation,
      'loss_function': loss_function,
      'optimizer':     optimizer,
    }

    # iterate all
    for target in ['activation', 'loss_function', 'optimizer']:

      # get param
      param = params[target]
      

      # if present in builtin, set
      if type(param) == str and param in self.builtin[target].keys():
        setattr(self, target, self.builtin[target][param]())

      # if object, set directly
      elif isinstance(param, object): setattr(self, target, param)
      
      # raise error
      # if not in {object, str} or string and not in builtin
      else: raise self.errors[target](param)


    #> Initial Weights & Bias

    # initial weights
    if isinstance(initial_weights, Number):                                            self.weights = np.repeat(initial_weights, input_size).astype(float)
    elif isinstance(initial_weights, Iterable) and len(initial_weights) == input_size: self.weights = np.array(initial_weights).astype(float)
    else: raise InvalidInitialWeights(f'Value must be either a number or an Iterable of size {input_size}')

    # initialize gradient to zero
    self.weight_gradient = np.repeat(0.0, self.input_size)

    # initial bias
    self.bias = initial_bias


  
  #
  # Forward
  #

  def forward(self, input):
    self.input = np.array(input)

    # if 1d, predict
    if len(self.input.shape) and self.input.shape[0] == self.input_size:
      # weighted sum
      weighted_sum = np.dot(self.input, self.weights) + self.bias

      # activation
      self.output = self.activation.apply(weighted_sum)

      # return
      return self.output
    
    # elif 2d, predict multiple
    return self.forward_multiple(self.input)



  #
  # Forward Multiple samples
  #

  def forward_multiple(self, inputs):
    inputs = np.array(inputs)

    if len(inputs.shape) < 2 or inputs.shape[1] != self.input_size:
      raise UnexpectedInputsShape(f'Inputs shape must be (n, {self.input_size}). found {inputs.shape}')

    results = []
    for inp in inputs:
      results.append(self.forward(inp))

    return np.array(results)



  #
  # Backward Pass (loss & optimize)
  #

  def backward(self, error=None):
    self.error = error or self.error

    # gradient
    self.delta = self.error * self.activation.derivative(self.output)

    # gradients for weights and bias
    self.weight_gradient = self.delta * self.input
    self.bias_gradient = self.delta

    # return loss for previous neurons
    return np.dot(self.delta, self.weights)


  def update(self, learning_rate=None):
    lr = learning_rate or self.learning_rate

    self.weights -= self.weight_gradient * lr
    self.bias    -= self.bias_gradient * lr


  #
  # Train
  #

  def train(self, inputs:Iterable, outputs:Iterable, epochs:int=100, batch_size:int=None, verbose:int=1):
    inputs = np.array(inputs)
    outputs = np.array(outputs)

    if len(inputs.shape) < 2 or inputs.shape[1] != self.input_size:    raise UnexpectedInputsShape (f'Inputs shape must be (n, {self.input_size}). found {inputs.shape}')
    if len(outputs.shape) != 1 or outputs.shape[0] != inputs.shape[0]: raise UnexpectedOutputsShape(f'Outputs size ({outputs.shape[0]}) differs from no of samples in input ({inputs.shape[0]})')

    # batch size: set counter
    if batch_size:
      self.batch_size = batch_size
      self.batch_counter = 0

    # batch reset
    def reset_batch(): self.predicted = []

    # batch end
    def batch_end():
      self.loss = self.loss_function.calculate(outputs, self.predicted)
      if verbose: print(f'epoch={e+1}, loss={self.loss}, samples={len(self.predicted)}')

    reset_batch()

    # main loop
    for e in range(epochs):
      
      if not batch_size: reset_batch()
      
      for i in range(outputs.shape[0]):

        # Forward Pass
        self.forward(inputs[i])
        self.predicted.append(self.output)

        # error calculation
        self.error = self.output - outputs[i]

        # Backward Pass
        self.backward()

        # Weights Updation
        if self.optimizer.mode == self.optimizer.MODES['SAMPLE']:
          self.optimizer.update(self)

        if verbose > 1: print(f'epoch={e+1}, sample={i+1}, actual={outputs[i]}, predicted={self.output}, error={self.error}')
      
        # batch logic
        if batch_size:
          self.batch_counter += 1
          if self.batch_counter >= self.batch_size:
            batch_end()

            if self.optimizer.mode == self.optimizer.MODES['BATCH']:
              self.optimizer.update(self)

            self.batch_counter = 0
            reset_batch()

      # epoch end (if batch_size=None)
      if not batch_size: batch_end()
    
    if batch_size and len(self.predicted):
      batch_end()



  #
  # Test
  #

  def test(self, inputs:Iterable, outputs:Iterable, verbose:int=0, binary:bool=True):
    inputs = np.array(inputs)
    outputs = np.array(outputs)

    if len(inputs.shape) < 2 or inputs.shape[1] != self.input_size:    raise UnexpectedInputsShape (f'Inputs shape must be (n, {self.input_size}). found {inputs.shape}')
    if len(outputs.shape) != 1 or outputs.shape[0] != inputs.shape[0]: raise UnexpectedOutputsShape(f'Outputs size ({outputs.shape[0]}) differs from no of samples in input ({inputs.shape[0]})')
    

    accurate = 0
    wrong    = 0
    for i in range(inputs.shape[0]):

      # Forward Pass
      predicted = self.forward(inputs[i])
      if binary: predicted = int(round(predicted))

      if predicted == outputs[i]: accurate+=1
      else:                       wrong   +=1

      if verbose > 1: print(f'sample={i+1}, predicted={predicted}, actual={outputs[i]}, error={outputs[i] - predicted}')

    return {
      'total':    accurate + wrong,
      'accurate': accurate,
      'wrong':    wrong,
    }


## Usage

In [50]:
# load dataset
from dataset import X, Y
X.shape

(10, 3)

#### Minimal

In [51]:
neuron = Neuron(3)

In [52]:
neuron.train(X, Y)

epoch=1, loss=0.4432049847947397, samples=10
epoch=2, loss=0.39826789813019803, samples=10
epoch=3, loss=0.38800732978082764, samples=10
epoch=4, loss=0.3782407475394864, samples=10
epoch=5, loss=0.36902108828904506, samples=10
epoch=6, loss=0.3603635683919453, samples=10
epoch=7, loss=0.35224540908069013, samples=10
epoch=8, loss=0.3446129230980055, samples=10
epoch=9, loss=0.3373906798764574, samples=10
epoch=10, loss=0.3304906628152556, samples=10
epoch=11, loss=0.32382023163393525, samples=10
epoch=12, loss=0.31728856326031585, samples=10
epoch=13, loss=0.31081178367288426, samples=10
epoch=14, loss=0.30431718570159794, samples=10
epoch=15, loss=0.2977468389299578, samples=10
epoch=16, loss=0.2910606577910456, samples=10
epoch=17, loss=0.28423872288841334, samples=10
epoch=18, loss=0.2772824550265156, samples=10
epoch=19, loss=0.2702141990468712, samples=10
epoch=20, loss=0.2630749157787413, samples=10
epoch=21, loss=0.255919979438577, samples=10
epoch=22, loss=0.2488134595671943, 

In [53]:
neuron.test(X, Y)

{'total': 10, 'accurate': 10, 'wrong': 0}

#### Detailed

In [54]:
neuron = Neuron(
  input_size=X.shape[1],
  learning_rate=0.1,
  activation=Sigmoid(),
  loss_function=MeanSquaredError(),
  optimizer=StochasticGradientDescent(),
  initial_weights=[0.1, 0.2, 0.3],
  initial_bias=0.5
)

In [55]:
neuron.train(
  inputs=X,
  outputs=Y,
  epochs=500,
  verbose=1
)

epoch=1, loss=0.43045228586346795, samples=10
epoch=2, loss=0.41836967798739455, samples=10
epoch=3, loss=0.4076938752225794, samples=10
epoch=4, loss=0.39745878576084726, samples=10
epoch=5, loss=0.38775760804506554, samples=10
epoch=6, loss=0.37864335775930424, samples=10
epoch=7, loss=0.37012215425377787, samples=10
epoch=8, loss=0.362158957738572, samples=10
epoch=9, loss=0.35468761575761687, samples=10
epoch=10, loss=0.3476219673920628, samples=10
epoch=11, loss=0.34086576928325546, samples=10
epoch=12, loss=0.33432053304942116, samples=10
epoch=13, loss=0.327891347332267, samples=10
epoch=14, loss=0.3214912442244972, samples=10
epoch=15, loss=0.3150447436109822, samples=10
epoch=16, loss=0.30849101789856626, samples=10
epoch=17, loss=0.3017867982154678, samples=10
epoch=18, loss=0.29490880594022206, samples=10
epoch=19, loss=0.28785524431357945, samples=10
epoch=20, loss=0.2806458091236319, samples=10
epoch=21, loss=0.2733198146185044, samples=10
epoch=22, loss=0.2659323521988818

In [56]:
neuron.weights

array([ 0.60686364, -0.23533579,  5.46429157])

In [57]:
neuron.bias

-4.010045078534286